In [1]:
!pip install pyspark
!pip install numpy
!pip install pandas
from pyspark.sql import SparkSession
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 45 kB/s 
     |████████████████████████████████| 199 kB 29.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=31ecb106732009d1b1205989b08e4afbb5cf59723716d74c572299aa4ff7fef0
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Create a SparkSession
spark = SparkSession.builder.appName("MyApp").getOrCreate()

In [3]:
#Load json file into dataframe 
from pyspark.sql.functions import from_json, to_json, col
from pyspark.sql import DataFrame

mydf = spark.read.json('/content/sample_data/outputfull.json')
mydf.show()
mydf.printSchema()

+-----------+--------------+--------+---------+--------------------+--------------------+--------------------+-----------+------+-----------+
| avg_rating|         brand|brand_id|   colour|         description|                name|        p_attributes|       p_id| price|ratingCount|
+-----------+--------------+--------+---------+--------------------+--------------------+--------------------+-----------+------+-----------+
|4.548826646|dupatta bazaar|   242.0|    White|White embroidered...|Dupatta Bazaar Wh...|{'Occasion': 'Dai...|  1518329.0| 899.0|     1321.0|
|4.536250816|dupatta bazaar|   242.0|    White|White solid dupat...|Dupatta Bazaar Wo...|{'Border': 'Tapin...|1.0711448E7| 599.0|     1531.0|
|4.366666667|dupatta bazaar|   242.0|   Orange|Orange and green ...|Dupatta Bazaar Or...|{'Border': 'Woven...|1.4964708E7| 899.0|       30.0|
|4.547413793|dupatta bazaar|   242.0|    Black|Black solid Dupat...|Dupatta Bazaar Bl...|{'Border': 'Solid...|1.3552234E7| 599.0|      232.0|
|     

In [4]:
workdf=mydf.select('*')
workdf=workdf.withColumn("p_id",workdf.p_id.cast('int'))
import pandas
#Create a temporary view from the data frame
workdf.createOrReplaceTempView("products")
result = spark.sql("SELECT * FROM products")
result.show()
result.printSchema()
result.select("p_attributes").dtypes
# to view p_attributes column and its contents
ffff = spark.sql("SELECT p_attributes FROM products")
ffff.show(truncate=False)




+-----------+--------------+--------+---------+--------------------+--------------------+--------------------+--------+------+-----------+
| avg_rating|         brand|brand_id|   colour|         description|                name|        p_attributes|    p_id| price|ratingCount|
+-----------+--------------+--------+---------+--------------------+--------------------+--------------------+--------+------+-----------+
|4.548826646|dupatta bazaar|   242.0|    White|White embroidered...|Dupatta Bazaar Wh...|{'Occasion': 'Dai...| 1518329| 899.0|     1321.0|
|4.536250816|dupatta bazaar|   242.0|    White|White solid dupat...|Dupatta Bazaar Wo...|{'Border': 'Tapin...|10711448| 599.0|     1531.0|
|4.366666667|dupatta bazaar|   242.0|   Orange|Orange and green ...|Dupatta Bazaar Or...|{'Border': 'Woven...|14964708| 899.0|       30.0|
|4.547413793|dupatta bazaar|   242.0|    Black|Black solid Dupat...|Dupatta Bazaar Bl...|{'Border': 'Solid...|13552234| 599.0|      232.0|
|        0.0|dupatta bazaar

In [9]:
#Since p_attributes column has key value pairs let us analyse it later. First let us draw some more common insights
# Query the table to find the products with the top 5  highest average ratings and low average ratings
highest_rated = spark.sql("SELECT brand,name,avg_rating FROM products ORDER BY avg_rating DESC LIMIT 5")
lowest_rated = spark.sql("SELECT brand,name,avg_rating FROM products ORDER BY avg_rating ASC LIMIT 5")
# Show the result
highest_rated.show(truncate=False)
lowest_rated.show(truncate=False)

+--------------+-----------------------------------------------------------------------------+----------+
|brand         |name                                                                         |avg_rating|
+--------------+-----------------------------------------------------------------------------+----------+
|dupatta bazaar|Dupatta Bazaar Blue & Green Printed Dupatta                                  |5.0       |
|roadster      |Roadster Women Olive Green Solid Joggers with Side Taping                    |5.0       |
|roadster      |Roadster Women Brown Solid Cargos Trousers                                   |5.0       |
|inddus        |Inddus Sea Green & Beige Embroidered Unstitched Lehenga & Blouse with Dupatta|5.0       |
|inddus        |Inddus Floral Poly Chiffon Draped Luxe Kurta Set                             |5.0       |
+--------------+-----------------------------------------------------------------------------+----------+

+--------------+-----------------------------

In [10]:
# To find the highly priced products, sort the data by the "price" column in descending order and get the top five rows
highest_priced = spark.sql("SELECT brand,name,price FROM products ORDER BY price DESC LIMIT 5")
#To find the lowest priced products, sort the data by the "price" column in ascending order and get the top five rows
lowest_priced = spark.sql("SELECT brand,name,price FROM products ORDER BY price ASC LIMIT 5")
highest_priced.show()
lowest_priced.show()

+-------------------+--------------------+-------+
|              brand|                name|  price|
+-------------------+--------------------+-------+
|     moksha designs|MOKSHA DESIGNS Ma...|47999.0|
|readiprint fashions|Readiprint Fashio...|33350.0|
|readiprint fashions|Readiprint Fashio...|30500.0|
|             shaily|Shaily Off-White ...|30090.0|
|readiprint fashions|Readiprint Fashio...|30000.0|
+-------------------+--------------------+-------+

+--------------------+--------------------+-----+
|               brand|                name|price|
+--------------------+--------------------+-----+
|                 max|max Peach-Coloure...|169.0|
|                 max|max White Regular...|199.0|
|                 max|max Cream Regular...|199.0|
|juniors by lifestyle|Juniors by Lifest...|249.0|
|              jockey|Jockey Women Blac...|295.0|
+--------------------+--------------------+-----+



In [11]:
# Find the most sold colour
most_sold_colour = spark.sql("SELECT colour, COUNT(*) as count FROM products GROUP BY colour ORDER BY count DESC LIMIT 1")

# Find the most selling brand
most_selling_brand = spark.sql("SELECT brand, COUNT(*) as count FROM products GROUP BY brand ORDER BY count DESC LIMIT 1")

# Find the most sold product(name)
most_sold_name = spark.sql("SELECT name, COUNT(*) as count FROM products GROUP BY name ORDER BY count DESC LIMIT 1")

# Find the most rated product ( product with highest number of ratings)
most_rated = spark.sql("SELECT name,ratingCount FROM products ORDER BY ratingCount DESC LIMIT 1")

most_sold_colour.show()
most_selling_brand.show()
most_sold_name.show(truncate=False)
most_rated.show(truncate=False)

+------+-----+
|colour|count|
+------+-----+
| Black| 1911|
+------+-----+

+--------+-----+
|   brand|count|
+--------+-----+
|roadster|  346|
+--------+-----+

+-----------------------------------+-----+
|name                               |count|
+-----------------------------------+-----+
|SCORPIOUS Women Pack of Shrug & Top|10   |
+-----------------------------------+-----+

+------------------------------------------------+-----------+
|name                                            |ratingCount|
+------------------------------------------------+-----------+
|AHIKA Women Black & Green Printed Straight Kurta|21274.0    |
+------------------------------------------------+-----------+



In [12]:
# Find the average rating and price for the top five brands
top_brands = spark.sql("SELECT brand, AVG(avg_rating) as average_rating FROM products GROUP BY brand ORDER BY average_rating DESC LIMIT 5")

# Show the results
top_brands.show(truncate=False)

+----------------------+--------------+
|brand                 |average_rating|
+----------------------+--------------+
|reebok                |4.833333333   |
|allen solly tribe     |4.769230769   |
|angel & rocket        |4.727272727   |
|kbz                   |4.677419355   |
|dexter by kook n keech|4.652173913   |
+----------------------+--------------+



In [13]:
from pyspark.sql.functions import corr

# To find out if there is any correlation between price and rating, compute the Pearson correlation coefficient between the "price" and "avg_rating" columns
correlation = workdf.select(corr("price", "avg_rating")).collect()[0][0]

# Print the result
print(correlation)

# With the below result, it is clear that the relation is weak and negative. Hence we do not need to consider.

-0.17412653741683132


In [14]:
print(workdf.schema.json())

{"fields":[{"metadata":{},"name":"avg_rating","nullable":true,"type":"double"},{"metadata":{},"name":"brand","nullable":true,"type":"string"},{"metadata":{},"name":"brand_id","nullable":true,"type":"double"},{"metadata":{},"name":"colour","nullable":true,"type":"string"},{"metadata":{},"name":"description","nullable":true,"type":"string"},{"metadata":{},"name":"name","nullable":true,"type":"string"},{"metadata":{},"name":"p_attributes","nullable":true,"type":"string"},{"metadata":{},"name":"p_id","nullable":true,"type":"integer"},{"metadata":{},"name":"price","nullable":true,"type":"double"},{"metadata":{},"name":"ratingCount","nullable":true,"type":"double"}],"type":"struct"}


In [15]:
from pyspark.sql.functions import col, get_json_object

superdf = workdf.select(
  col("p_id"),
  col("name"),
  col("price"),
  col("colour"),
  col("brand"),
  col("ratingCount"),
  col("avg_rating"),
  get_json_object(col("p_attributes"), "$.Occasion").alias("Occasion"),
  get_json_object(col("p_attributes"), "$.Border").alias("Border"),
  get_json_object(col("p_attributes"), "$.Fabric").alias("Fabric"),
  get_json_object(col("p_attributes"), "$.Ornamentation").alias("Ornamentation"),
  get_json_object(col("p_attributes"), "$.Pattern").alias("Pattern"),
  get_json_object(col("p_attributes"), "$.Print or Pattern Type").alias("Print or Pattern Type"),
  get_json_object(col("p_attributes"), "$.Sustainable").alias("Sustainable"),
  get_json_object(col("p_attributes"), "$.Technique").alias("Technique"),
  get_json_object(col("p_attributes"), "$.Wash Care").alias("Wash Care"),
  get_json_object(col("p_attributes"), "$.Wedding").alias("Wedding"),
  get_json_object(col("p_attributes"), "$.Add-Ons").alias("AddOns")
)

superdf.show()

+--------+--------------------+------+---------+--------------+-----------+-----------+--------+------------+-----------------+-------------+------------+---------------------+-----------+-----------+------------+--------------------+------+
|    p_id|                name| price|   colour|         brand|ratingCount| avg_rating|Occasion|      Border|           Fabric|Ornamentation|     Pattern|Print or Pattern Type|Sustainable|  Technique|   Wash Care|             Wedding|AddOns|
+--------+--------------------+------+---------+--------------+-----------+-----------+--------+------------+-----------------+-------------+------------+---------------------+-----------+-----------+------------+--------------------+------+
| 1518329|Dupatta Bazaar Wh...| 899.0|    White|dupatta bazaar|     1321.0|4.548826646|   Daily|        null|             null|         null| Embroidered|               Floral|       null|       null|        null|                null|  null|
|10711448|Dupatta Bazaar Wo...| 

In [16]:
#Create a temporary view from the super data frame
superdf.createOrReplaceTempView("superproducts")
superresult = spark.sql("SELECT * FROM superproducts")
superresult.show()
superresult.printSchema()

+--------+--------------------+------+---------+--------------+-----------+-----------+--------+------------+-----------------+-------------+------------+---------------------+-----------+-----------+------------+--------------------+------+
|    p_id|                name| price|   colour|         brand|ratingCount| avg_rating|Occasion|      Border|           Fabric|Ornamentation|     Pattern|Print or Pattern Type|Sustainable|  Technique|   Wash Care|             Wedding|AddOns|
+--------+--------------------+------+---------+--------------+-----------+-----------+--------+------------+-----------------+-------------+------------+---------------------+-----------+-----------+------------+--------------------+------+
| 1518329|Dupatta Bazaar Wh...| 899.0|    White|dupatta bazaar|     1321.0|4.548826646|   Daily|        null|             null|         null| Embroidered|               Floral|       null|       null|        null|                null|  null|
|10711448|Dupatta Bazaar Wo...| 

In [18]:
# Find the table with Add-Ons
Addonstable = spark.sql("SELECT name,AddOns FROM superproducts WHERE AddOns!='NA'")
Addonstable.show()

+--------------------+--------------------+
|                name|              AddOns|
+--------------------+--------------------+
|The Roadster Life...|   Comes with a belt|
|Roadster Women Bl...|Comes with an acc...|
|Roadster Women De...|   Comes with a belt|
|Roadster Women Bl...|   Comes with a belt|
|The Roadster Life...|   Comes with a belt|
|Roadster Women Bl...|   Comes with a belt|
|Roadster Women Bl...|   Comes with a belt|
|The Roadster Life...|   Comes with a belt|
|Roadster Women Al...|   Comes with a belt|
|Roadster Women Bl...|                Belt|
|Inddus Women Attr...|              Jacket|
|Inddus Women Must...|              Jacket|
|Inddus Women Eleg...|              Jacket|
|Inddus Women Blue...|              Jacket|
|Inddus Women Burg...|              Jacket|
|SASSAFRAS Burgund...|   Comes with a belt|
|SASSAFRAS Women T...|                Belt|
|Kotty Women Peach...|   Comes with a belt|
|Kotty Women Purpl...|   Comes with a belt|
|KASSUALLY Women C...|   Comes w

In [ ]:
# Find the most sold clothes as per occasion
most_sold_occasion = spark.sql("SELECT Occasion, COUNT(*) as count FROM superproducts GROUP BY Occasion ORDER BY count DESC LIMIT 5")

# Find the most selling frabic
most_selling_fabric = spark.sql("SELECT Fabric, COUNT(*) as count FROM superproducts GROUP BY Fabric ORDER BY count DESC LIMIT 5")

# Find the most selling pattern

most_selling_pattern = spark.sql("SELECT Pattern, COUNT(*) as count FROM superproducts GROUP BY Pattern ORDER BY count DESC LIMIT 5")

most_sold_occasion.show()
most_selling_fabric.show()
most_selling_pattern.show()

+--------+-----+
|Occasion|count|
+--------+-----+
|  Casual| 7500|
|   Daily| 1637|
| Festive| 1130|
|    null| 1099|
|  Ethnic| 1066|
+--------+-----+

+-------------+-----+
|       Fabric|count|
+-------------+-----+
|         null| 3880|
|       Cotton| 3270|
|    Polyester| 2651|
|Viscose Rayon| 1353|
|  Pure Cotton|  626|
+-------------+-----+

+------------+-----+
|     Pattern|count|
+------------+-----+
|       Solid| 4550|
|        null| 3805|
|     Printed| 2956|
| Embroidered|  698|
|Woven Design|  559|
+------------+-----+



In [ ]:

superdf.toPandas().to_csv('analysisoutput.csv')
